#  Baby names mini-project

### Import  libraries:

In [201]:
import altair as alt
import pandas as pd
import geopandas as gpd 
from IPython.display import display

alt.data_transformers.enable('json') 

from ipywidgets import interact, interactive, fixed, interact_manual,Dropdown
import ipywidgets as widgets

pass

### Reading data:

In [202]:
#### code taken form altair tutorial

#national dataset
na_data = pd.DataFrame(pd.read_csv("nat-data.tsv", sep="\t"))
na_data.drop(na_data[na_data.preusuel == '_PRENOMS_RARES'].index, inplace=True)
na_data.drop(na_data[na_data.annais == 'XXXX'].index, inplace=True)

# dataset per departement
names = pd.read_csv("dpt2020.csv", sep=";")
names.drop(names[names.preusuel == '_PRENOMS_RARES'].index, inplace=True)
names.drop(names[names.dpt == 'XX'].index, inplace=True)
names.drop(names[names.dpt == 'NAN'].index, inplace=True)
names.sample(10)


,sexe,preusuel,annais,dpt,nombre
2101527,2,CÉLIA,1987,57,21
2443048,2,FELICITE,1906,59,5
3291093,2,NANCY,1974,54,12
673457,1,GUY,1933,36,56
151704,1,ANTONIO,1951,69,5
352266,1,DANIEL,1938,83,19
3252428,2,MORGANE,1971,75,10
2886148,2,LOLA,1999,29,9
2348254,2,ELORRI,1983,64,3
231318,1,BERNARD,1948,25,166


In [203]:
# Just to test visualisation
#na_data=na_data[50999:150000]
na_data.shape

(586591, 4)

### DATA MANIPULATION:

###### National data

In [204]:
List_name=na_data['preusuel'].unique()

List_year=na_data['annais'].unique()

nat_data_name={}
    
List_year.sort()
List_year[0]

'1900'

In [205]:

#distribution by year
for i in range(len(List_year)):
    nat_data_name[List_year[i]]=na_data.loc[na_data['annais'] == List_year[i]]
    

#### Department data

In [206]:
year_data_name={}

List_dep=names['dpt'].unique()

#distribution by year
for i in range(len(List_year)):
    year_data_name[List_year[i]]=names.loc[names['annais'] == List_year[i]]


In [207]:
# List_name_per_dep=dep_data_name['84']['preusuel'].unique()
# List_year_per_dep=dep_data_name['84']['annais'].unique()
# List_year_per_dep

dep_year_popular_name={}
sub_dp_year_popular_name={}
sub_sub_dp_year_popular_name={}

for i in range(len(List_year)):
    dep_year_popular_name[List_year[i]]=[]
    for j in List_dep:
        sub_dp_year_popular_name[j]=year_data_name[List_year[i]].loc[year_data_name[List_year[i]]['dpt'] ==j]
        dep_year_popular_name[List_year[i]].append(sub_dp_year_popular_name[j])

In [100]:
### find the 10 popular names in each year

nat_data_name_popular={}  #dictionnary of the 10 popular names for each year
nat_exten_name_popular={} #dictionnary of the 50 popular names for each year
    
    
for i in range(len(List_year)):
     nat_data_name[List_year[i]]=nat_data_name[List_year[i]].sort_values(by=['nombre'],ascending=False)
     nat_data_name_popular[List_year[i]]=nat_data_name[List_year[i]][:10]
     nat_exten_name_popular[List_year[i]]=nat_data_name[List_year[i]][:50]
print(list(nat_data_name_popular['1920']['preusuel'])) #perfect  

['MARIE', 'JEAN', 'ANDRÉ', 'JEANNE', 'PIERRE', 'RENÉ', 'MARCEL', 'ROGER', 'LOUIS', 'HENRI']


## Second visualisation

In [208]:
# departement data

departement = gpd.read_file('departements-version-simplifiee.geojson')

departement.sample(5)

,code,nom,geometry
23,25,Doubs,"POLYGON ((6.80701 47.56280, 6.81666 47.54792, ..."
36,36,Indre,"POLYGON ((1.32667 47.18623, 1.40143 47.21245, ..."
90,90,Territoire de Belfort,"POLYGON ((6.82354 47.81305, 6.84618 47.82295, ..."
71,71,Saône-et-Loire,"POLYGON ((4.11597 47.12334, 4.15377 47.11456, ..."
77,77,Seine-et-Marne,"POLYGON ((2.57166 48.69201, 2.56880 48.70722, ..."


In [209]:

names_after_merge = departement.merge(names, how='right', left_on='code', right_on='dpt')

names_after_merge.sample(5)

,code,nom,geometry,sexe,preusuel,annais,dpt,nombre
3568216,78,Yvelines,"POLYGON ((2.20059 48.90868, 2.16838 48.89508, ...",2,TANIA,1989,78,8
2921424,02,Aisne,"POLYGON ((4.04797 49.40564, 4.03991 49.39740, ...",2,MADISON,1996,02,4
1061000,94,Val-de-Marne,"POLYGON ((2.33190 48.81701, 2.36395 48.81632, ...",1,MANUEL,1974,94,21
3643767,88,Vosges,"POLYGON ((5.47006 48.42093, 5.51099 48.41822, ...",2,YOLANDE,1952,88,23
3192393,79,Deux-Sèvres,"POLYGON ((-0.89196 46.97582, -0.85592 46.97908...",2,MOISETTE,1939,79,3


In [211]:
#find the popular name for a given year and departement


def popular_name_dep(year,departement):
    dpt=''
    pop_name='Not defined'
    if year in List_year:
        for i in range(len(dep_year_popular_name[year])):
            df_popular_name=dep_year_popular_name[year][i].sort_values(by=['nombre'],ascending=False)
            if df_popular_name[:1].empty:
                continue
            else:
                dpt=df_popular_name[:1]["dpt"].values[0]
                if departement==dpt:
                    pop_name=df_popular_name[:1]["preusuel"].values[0]
    
    return pop_name

        

In [212]:
year="2000"
dep_year_pop =pd.DataFrame(List_dep, columns =['dpt'])
dep_year_pop["Pop_name"]=[1]*len(List_dep)
for i in dep_year_pop.index :
    dep_year_pop.at[i,'Pop_name']=popular_name_dep(year,dep_year_pop['dpt'].values[i])

#no_subset=departement.merge(dep_year_pop, how='right', left_on='code', right_on='dpt')

In [213]:
dr_List_year=Dropdown(options=List_year[::-1])
li=list(nat_exten_name_popular['2017']['preusuel'])
li.insert(0,'.....')
dr_name=Dropdown(options=li)

def update(*args):
    selected_year=dr_List_year.value
    li=list(nat_exten_name_popular[dr_List_year.value]['preusuel'])
    li.insert(0,'.....')
    dr_name.options = list(li)
dr_List_year.observe(update)

             
def make_altair_geo_chart(year,name):
      big_subset=[]
      if(name=='.....'):
        dep_year_pop =pd.DataFrame(List_dep, columns =['dpt'])
        dep_year_pop["Pop_name"]=[1]*len(List_dep)
        for i in dep_year_pop.index :
            dep_year_pop.at[i,'Pop_name']=popular_name_dep(year,dep_year_pop['dpt'].values[i])
        n_subset=departement.merge(dep_year_pop, how='right', left_on='code', right_on='dpt')
        my_chart=alt.Chart(n_subset).mark_geoshape(stroke='white').encode(
        tooltip=['nom:N', 'code:N', 'Pop_name:N'],
        color='Pop_name',
        ).properties(width=800, height=600)
      else:
        na_grouped = names.groupby(['dpt', 'preusuel', 'sexe','annais'], as_index=False).sum()
        grouped=na_grouped[na_grouped.annais==year] # filter data by selected year
        grouped = departement.merge(grouped, how='right', left_on='code', right_on='dpt')
        d_subset=grouped[grouped.preusuel == name]   # filter data by selected name
        my_chart=alt.Chart(d_subset).mark_geoshape(stroke='white').encode(
        tooltip=['preusuel:N','nom:N', 'code:N', 'nombre:N'],
        color='nombre',
        ).properties(width=800, height=600)
        
      return my_chart
        
     
      
  

interact(make_altair_geo_chart, year=dr_List_year, name=dr_name)

interactive(children=(Dropdown(description='year', options=('2017', '2016', '2015', '2014', '2013', '2012', '2…

<function __main__.make_altair_geo_chart(year, name)>